# Исследование надежности заемщиков


In [1]:
import pandas as pd
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных <a id='category'></a>

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

In [22]:
df_children = data.pivot_table(index ='children',values = 'debt',aggfunc = ['count','mean'])

df_children['debt%'] = df_children['mean']['debt']*100
df_children = df_children.sort_values(by='debt%',ascending = False)
display(df_children)

#Столбец `debt` показывает имел ли клиент задолженность по возврату кредитов.

#Создадим сводную таблицу, чтобы увидеть есть ли зависимость среднего значения должников от количества детей

,count,mean,debt%
,debt,debt,
children,,,
4,41,0.097561,9.756098
2,2052,0.094542,9.454191
1,4809,0.092327,9.232689
3,330,0.081818,8.181818
0,14107,0.075353,7.535266
5,9,0.000000,0.000000


In [23]:
data['children_group'] = data['children'].apply(lambda x: 'Нет детей' if x == 0 else 'Дети есть')

df_example = data.pivot_table(index = 'children_group', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children_group,,,,
Дети есть,7241,669,9.24%,90.76%
Нет детей,14107,1063,7.54%,92.46%


**Вывод:** 
Можно увидеть совсем небольшую зависимость так как процентные отношения должников к общему кол-ву клиентов практически одинаковый во всех подгруппах. Но основываясь на полученных средних значениях должников можно сказать, что самыми потенциально опасными клиентами являются клиент с 4-мя детьми

In [24]:
df_fam_stat = data.pivot_table(index ='family_status',values = 'debt',aggfunc = ['count','mean'])

df_fam_stat['debt%'] = df_fam_stat['mean']['debt']*100
df_fam_stat = df_fam_stat.sort_values(by='debt%',ascending = False)
display(df_fam_stat)



#Столбец `family_status` показывает семейное положение клиента.

#Создадим сводную таблицу, чтобы увидеть есть ли зависимость среднего значения должников от семейного положения


,count,mean,debt%
,debt,debt,
family_status,,,
Не женат / не замужем,2796,0.097639,9.763948
гражданский брак,4146,0.092861,9.286059
женат / замужем,12266,0.075575,7.557476
в разводе,1189,0.070648,7.064760
вдовец / вдова,951,0.066246,6.624606


**Вывод:** 
Можно увидеть совсем небольшую зависимость так как процентные отношения должников к общему кол-ву клиентов практически одинаковый во всех подгруппах. Но самые потенциально опасные клиенты, которые не выплатят кредит в срок - не женаты/не замужем

In [25]:
df_total_income = data.pivot_table(index ='total_income_category',values = 'debt',aggfunc = ['count','mean'])

df_total_income['debt%'] = df_total_income['mean']['debt']*100
df_total_income = df_total_income.sort_values(by='debt%',ascending = False)
display(df_total_income)
#Столбец `total_income_category` показывает категорию заработка клиента, которыя была ранее описана выше.

#Создадим сводную таблицу, чтобы увидеть есть ли зависимость среднего значения должников от категории заработка клиента


,count,mean,debt%
,debt,debt,
total_income_category,,,
E,22,0.090909,9.090909
C,15938,0.084891,8.489145
A,25,0.080000,8.000000
B,5014,0.070602,7.060231
D,349,0.060172,6.017192


**Вывод:** 
Здесь также практически не наблюдается никакой зависимости из-за малых отклонений значений друг от друга. Но наиболее вероятно что клиенты с категорией заработка `D` вернут кредит в срок, относительно остальных клиентов

In [26]:
df_purpose = data.pivot_table(index ='purpose_category',values = 'debt',aggfunc = ['count','mean'])

df_purpose['debt%'] = df_purpose['mean']['debt']*100
df_purpose = df_purpose.sort_values(by='debt%',ascending = False)
display(df_purpose)

#Столбец `purpose_category` показывает цель кредита клиента.

#Создадим сводную таблицу, чтобы увидеть есть ли зависимость среднего значения должников от цели кредита клиента

,count,mean,debt%
,debt,debt,
purpose_category,,,
операции с автомобилем,4281,0.093436,9.343611
получение образования,3989,0.092504,9.250439
проведение свадьбы,2324,0.078744,7.874355
операции с недвижимостью,10754,0.072531,7.253115


**Вывод:** 
В итоге можно увидеть, что клиенты, цель которых "Операции с автомобилем" и "Получение образования" вероятнее всего не выплатят кредит в срок

*Ответ:* 
 - Все мы люди, а у людей есть человеческий фактор, который иногда создаёт подобного рода ошибки
 - Технические ошибки, вызванные непосредственно машиной

*Ответ:* 
Это связано с тем, что у нас может быть ряд примерно схожих по порядку значений, но также в этом ряду могут присутствовать и несколько значений , которые на несколько порядков больше остальных, соответственно, если мы будем использовать среднее арифметическое значение, то получим нереалистичные завышенные результаты в сравнении с медианным значением

## Общий вывод.

Были получены следующие зависимости между:
- Количеством детей от возврата кредита в срок
- Семейным положением от возврата кредита в срок
- Уровнем дохода от возврата кредита в срок
- Целей кредита от возврата кредита в срок

Для того чтобы избежать рисков компании следует стараться с большей осторожностью выдавать кредит следующим типам людей:
 - Не женатым / не замужним
 - С маленьким уровнем заработка ('Категория E')
 - Имеющие детей
 - Цель кредита которых - Операции с автомобилем / Образованием